In [5]:
import os


In [6]:
%pwd

'e:\\Projects\\Text-Summarizer-Project\\research'

In [7]:
os.chdir("../")

In [8]:
%pwd

'e:\\Projects\\Text-Summarizer-Project'

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataInjestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [10]:
%pip install -e ../

Obtaining file:///E:/Projects
Note: you may need to restart the kernel to use updated packages.


ERROR: file:///E:/Projects does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [11]:
from setuptools import find_packages

packages = find_packages("src")
package_dir = {"": "src"}


In [12]:
import sys
sys.path.append("/e/Projects/Text-Summarizer-Project/src")



In [16]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [32]:
from pathlib import Path  # Make sure this is imported if you're using Path objects

class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        # Convert to string before passing to read_yaml
        self.config = read_yaml(str(config_filepath))
        self.params = read_yaml(str(params_filepath))
        
        create_directories([self.config.artifacts_root])
                
    def get_data_ingestion_config(self) -> DataInjestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataInjestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = Path(config.local_data_file),
            unzip_dir = Path(config.unzip_dir)
        )
        return data_ingestion_config

In [27]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size


In [23]:
class DataInjestion:
    def __init__(self, config: DataInjestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
           filename, headers = request.urlretrieve(
               url = self.config.source_URL,
               filename = self.config.local_data_file
           )
           logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"File already exists with size: {get_size(Path(self.config.local_data_file))}")
            
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [31]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataInjestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f"An error occurred: {e}")
    raise e

[2025-05-22 21:27:15,780: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-05-22 21:27:15,921: INFO: common: YAML file: params.yaml loaded successfully]
[2025-05-22 21:27:15,951: INFO: common: Created directory at: artifacts]
[2025-05-22 21:27:15,955: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-05-22 21:27:15,957: ERROR: 3373676244: An error occurred: name 'Pathconfig' is not defined]


NameError: name 'Pathconfig' is not defined